In [1]:
# Import helps
from src.helpers import *
from src.plots import *

# Import base
import pandas as pd
import numpy as np

# Preprocessing
from sklearn.model_selection import train_test_split

# Feature selection
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from umap import UMAP
from sklearn.pipeline import Pipeline

# tensorflow
import tensorflow as tf

import sys

pd.options.display.max_rows = 999

# Set seeds for reproducibility
set_seeds()

2024-01-04 12:55:10.782714: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-04 12:55:10.810102: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-04 12:55:10.810123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-04 12:55:10.810925: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-04 12:55:10.815494: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Reload the saved scaled data
X_train = pd.read_csv('data/train/scaled_X_train.csv', parse_dates=True, index_col='unix')
y_train = pd.read_csv('data/train/y_train.csv', parse_dates=True, index_col='unix')
X_test = pd.read_csv('data/test/scaled_X_test.csv', parse_dates=True, index_col='unix')
y_test = pd.read_csv('data/test/y_test.csv', parse_dates=True, index_col='unix')
X_val = pd.read_csv('data/val/scaled_X_val.csv', parse_dates=True, index_col='unix')
y_val = pd.read_csv('data/val/y_val.csv', parse_dates=True, index_col='unix')

In [7]:
y_train.iloc[6:].head(128)

,label
unix,
2018-02-26 15:00:00+00:00,0
2018-02-26 16:00:00+00:00,1
2018-02-26 17:00:00+00:00,0
2018-02-26 18:00:00+00:00,0
2018-02-26 19:00:00+00:00,0
2018-02-26 20:00:00+00:00,1
2018-02-26 21:00:00+00:00,0
2018-02-26 22:00:00+00:00,0
2018-02-26 23:00:00+00:00,1


In [3]:
# Reshape the data into the correct format
seqlen = 6
featurelen = X_train.shape[-1]
train_tensors = tf.keras.utils.timeseries_dataset_from_array(X_train, y_train.iloc[seqlen:], seqlen)
val_tensors = tf.keras.utils.timeseries_dataset_from_array(X_val, y_val.iloc[seqlen:], seqlen)
test_tensors = tf.keras.utils.timeseries_dataset_from_array(X_test, y_test.iloc[seqlen:], seqlen)

2024-01-04 12:55:17.075592: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-04 12:55:17.098188: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-04 12:55:17.098315: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
for element in train_tensors:
    print(element)

(<tf.Tensor: shape=(128, 6, 324), dtype=float64, numpy=
array([[[ 2.67696020e-02,  1.36956013e+00,  1.08761160e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 3.22273833e-01,  1.43484671e+00,  1.41199602e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.12714221e+00,  1.36998893e+00,  1.37109438e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 5.55145587e-01,  1.43465005e+00,  1.84358898e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 2.47307743e+00,  1.44008770e+00,  8.67426470e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 3.47012496e+00,  1.43546358e+00, -5.09497214e-02, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 3.22273833e-01,  1.43484671e+00,  1.41199602e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.12714221e+00,  1.36998893e+00,  1.37109438e+00, 